<a href="https://colab.research.google.com/github/sadiksmart0/Bank-Marketing/blob/main/Bank_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

##Bank Marketing Data Set
##Download: Data Folder, Data Set Description

##Abstract: The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution.
#The classification goal is to predict if the client will subscribe a term deposit (variable y).
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#IMPORTING DEPENDENCIES
import pandas as pd
from pyspark.sql import SparkSession
import seaborn as sns
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.sql.functions import col
from pyspark.ml.feature import StandardScaler

In [3]:
import mlflow

In [4]:
# STARTING A SPARK SESSION
spark = SparkSession.builder.getOrCreate()

In [5]:
# MOUNTING DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Loading Dataset
df1 = pd.read_csv("/content/drive/MyDrive/bank-full.csv", sep=";")

In [7]:
# Number of columns
len(df1.columns)

17

In [8]:
df1.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [9]:
# Unique Counts of Subscribtion
df1["y"].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

In [10]:
# Slicing 
df_yes = df1[df1["y"]=="yes"]
df_no = df1[df1["y"]=="no"]
len(df_yes)

5289

In [11]:
# DOWNSAMPLING OF IMBALANCED DATASET
downsampled = df_no.sample(n=len(df_yes), replace=False, random_state=1)

In [12]:
# CONCATENATING 
bank_data = pd.concat([df_yes, downsampled])
# SHUFFLING 
shuffled_bank_data = bank_data.sample(frac=1, random_state=21)
len(bank_data)

10578

In [13]:
# DROP UNWANTED COLUMNS
bank_final = shuffled_bank_data.drop(["month", "day", "contact"], axis=1)
len(bank_final.columns)

14

In [14]:
# LOADING SPARK DATAFRAME
bank_df = spark.createDataFrame(bank_final)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [15]:
# VIEW TOP 10
bank_df.show(10)

+---+-------------+--------+---------+-------+-------+-------+----+--------+--------+-----+--------+--------+---+
|age|          job| marital|education|default|balance|housing|loan|duration|campaign|pdays|previous|poutcome|  y|
+---+-------------+--------+---------+-------+-------+-------+----+--------+--------+-----+--------+--------+---+
| 67|    housemaid|divorced|secondary|     no|   5275|     no|  no|     222|       4|   -1|       0| unknown|yes|
| 34|   technician| married|secondary|     no|    545|    yes|  no|      96|       4|   -1|       0| unknown| no|
| 29|self-employed|  single| tertiary|     no|   1579|     no|  no|     190|       1|   92|       5| success|yes|
| 36|  blue-collar|  single|secondary|     no|    366|    yes| yes|    1133|       2|   -1|       0| unknown|yes|
| 81|      retired|divorced|  primary|     no|    949|     no|  no|     188|       2|  280|       1|   other|yes|
| 34|   technician| married|secondary|     no|    656|    yes|  no|      10|       7|  3

In [16]:
# DROP EMPTY ROWS
bank_final = bank_df.dropna(how="any")

In [17]:
# VIEW SCHEMA
bank_final.printSchema()

root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



#ONE HOT ENCODING CATEGORICAL COLUMNS


In [18]:
# Selecting of input and Output Columns
inputCols = ['job', 'marital', 'education', 'default','housing', 'loan', 'poutcome','y']
outputCols = [col+"_encoded" for col in inputCols]

# Create separate StringIndexer instances for each input column
indexers = [StringIndexer(inputCol=col, outputCol=col+"_Index") for col in inputCols]

# Fit and transform the DataFrame using the indexers
indexed = bank_final
for indexer in indexers:
    indexed = indexer.fit(indexed).transform(indexed)

# Create separate OneHotEncoder instances for each input column
encoders = [OneHotEncoder(inputCol=col+"_Index", outputCol=outputCols[i]) for i, col in enumerate(inputCols)]

# Fit and transform the DataFrame using the encoders
encoded = indexed
for encoder in encoders:
    encoded = encoder.fit(encoded).transform(encoded)

# # # Drop the indexed columns
for col in inputCols:
      if col != "y":
        encoded = encoded.drop(col+"_Index")
        encoded = encoded.drop(col)

# Display the transformed DataFrame
encoded.show()

+---+-------+--------+--------+-----+--------+---+-------+---------------+---------------+-----------------+---------------+---------------+-------------+----------------+-------------+
|age|balance|duration|campaign|pdays|previous|  y|y_Index|    job_encoded|marital_encoded|education_encoded|default_encoded|housing_encoded| loan_encoded|poutcome_encoded|    y_encoded|
+---+-------+--------+--------+-----+--------+---+-------+---------------+---------------+-----------------+---------------+---------------+-------------+----------------+-------------+
| 67|   5275|     222|       4|   -1|       0|yes|    1.0|(11,[10],[1.0])|      (2,[],[])|    (3,[0],[1.0])|  (1,[0],[1.0])|  (1,[0],[1.0])|(1,[0],[1.0])|   (3,[0],[1.0])|    (1,[],[])|
| 34|    545|      96|       4|   -1|       0| no|    0.0| (11,[2],[1.0])|  (2,[0],[1.0])|    (3,[0],[1.0])|  (1,[0],[1.0])|      (1,[],[])|(1,[0],[1.0])|   (3,[0],[1.0])|(1,[0],[1.0])|
| 29|   1579|     190|       1|   92|       5|yes|    1.0| (11,[7],[1.

#SCALING 

In [19]:
encoded.printSchema()

root
 |-- age: long (nullable = true)
 |-- balance: long (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- y: string (nullable = true)
 |-- y_Index: double (nullable = false)
 |-- job_encoded: vector (nullable = true)
 |-- marital_encoded: vector (nullable = true)
 |-- education_encoded: vector (nullable = true)
 |-- default_encoded: vector (nullable = true)
 |-- housing_encoded: vector (nullable = true)
 |-- loan_encoded: vector (nullable = true)
 |-- poutcome_encoded: vector (nullable = true)
 |-- y_encoded: vector (nullable = true)



In [20]:
input_cols = ["balance","duration","pdays"]
output_cols = [col+"scaled" for col in input_cols]


assembler = VectorAssembler(inputCols=input_cols, outputCol='s_features')
encoded = assembler.transform(encoded)

scaler = StandardScaler(inputCol="s_features", outputCol="scaled_features",
                          withStd=True, withMean=False)
scaled = scaler.fit(encoded)
encoded = scaled.transform(encoded)



In [21]:
encoded.show(5)

+---+-------+--------+--------+-----+--------+---+-------+---------------+---------------+-----------------+---------------+---------------+-------------+----------------+-------------+-------------------+--------------------+
|age|balance|duration|campaign|pdays|previous|  y|y_Index|    job_encoded|marital_encoded|education_encoded|default_encoded|housing_encoded| loan_encoded|poutcome_encoded|    y_encoded|         s_features|     scaled_features|
+---+-------+--------+--------+-----+--------+---+-------+---------------+---------------+-----------------+---------------+---------------+-------------+----------------+-------------+-------------------+--------------------+
| 67|   5275|     222|       4|   -1|       0|yes|    1.0|(11,[10],[1.0])|      (2,[],[])|    (3,[0],[1.0])|  (1,[0],[1.0])|  (1,[0],[1.0])|(1,[0],[1.0])|   (3,[0],[1.0])|    (1,[],[])|[5275.0,222.0,-1.0]|[1.52399583141426...|
| 34|    545|      96|       4|   -1|       0| no|    0.0| (11,[2],[1.0])|  (2,[0],[1.0])|  

In [22]:
# Feature and Target
unwanted = ["balance","duration","pdays","y_encoded","y","s_features"]

for col in unwanted:
    features = encoded.drop(col)
    encoded = features
features.show(2)

+---+--------+--------+-------+---------------+---------------+-----------------+---------------+---------------+-------------+----------------+--------------------+
|age|campaign|previous|y_Index|    job_encoded|marital_encoded|education_encoded|default_encoded|housing_encoded| loan_encoded|poutcome_encoded|     scaled_features|
+---+--------+--------+-------+---------------+---------------+-----------------+---------------+---------------+-------------+----------------+--------------------+
| 67|       4|       0|    1.0|(11,[10],[1.0])|      (2,[],[])|    (3,[0],[1.0])|  (1,[0],[1.0])|  (1,[0],[1.0])|(1,[0],[1.0])|   (3,[0],[1.0])|[1.52399583141426...|
| 34|       4|       0|    0.0| (11,[2],[1.0])|  (2,[0],[1.0])|    (3,[0],[1.0])|  (1,[0],[1.0])|      (1,[],[])|(1,[0],[1.0])|   (3,[0],[1.0])|[0.15745549348261...|
+---+--------+--------+-------+---------------+---------------+-----------------+---------------+---------------+-------------+----------------+--------------------+
only

In [23]:
encoded.columns
combined = ['age','campaign','previous','job_encoded','marital_encoded','education_encoded','default_encoded','housing_encoded','loan_encoded','poutcome_encoded','scaled_features']

# SPLITTING TRAIN-TEST DATA

In [24]:

# Split the data into training and testing sets
train_data, test_data = encoded.randomSplit([0.7, 0.3], seed=42)

# Combine all features into one Vector
assembler = VectorAssembler(inputCols=combined, outputCol='all_features')

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

train_data.printSchema()

root
 |-- age: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- y_Index: double (nullable = false)
 |-- job_encoded: vector (nullable = true)
 |-- marital_encoded: vector (nullable = true)
 |-- education_encoded: vector (nullable = true)
 |-- default_encoded: vector (nullable = true)
 |-- housing_encoded: vector (nullable = true)
 |-- loan_encoded: vector (nullable = true)
 |-- poutcome_encoded: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- all_features: vector (nullable = true)



# Modelization

In [25]:
from mlflow.pyspark.ml import autolog
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Declare hyperparameters
max_iters = [50, 100, 200]
thresholds = [0.3, 0.5, 0.7]
random_states = [42, 123, 456]

# Set the tracking uri
# mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Start an MLflow experiment
mlflow.set_experiment("Logistic_Regression")

# Enable automatic logging
autolog()

# Start a loop to iterate over hyperparameters
for i in range(len(max_iters)):

    # Start an MLflow run for the current set of hyperparameters
    with mlflow.start_run():

        # Declare model
        lr = LogisticRegression(featuresCol="all_features", labelCol="y_Index", maxIter=max_iters[i], threshold=thresholds[i])

        # Model Training
        lr_model = lr.fit(train_data)

        # Predictions
        predictions = lr_model.transform(test_data)

        # Evaluate the model using a binary classification evaluator
        # Accuracy
        evaluator = BinaryClassificationEvaluator(labelCol="y_Index", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
        accuracy = evaluator.evaluate(predictions)
        # Recall
        evaluator = MulticlassClassificationEvaluator(labelCol="y_Index", predictionCol="prediction", metricName="weightedRecall")
        recall = evaluator.evaluate(predictions)
        # Precision
        evaluator = MulticlassClassificationEvaluator(labelCol="y_Index", predictionCol="prediction", metricName="weightedPrecision")
        precision = evaluator.evaluate(predictions)

        # Compute the confusion matrix
        confusion_matrix = predictions.groupBy("y_Index", "prediction").count().orderBy("y_Index", "prediction")



2023/04/13 14:38:30 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('job_encoded', VectorUDT(), True), StructField('marital_encoded', VectorUDT(), True), StructField('education_encoded', VectorUDT(), True), StructField('default_encoded', VectorUDT(), True), StructField('housing_encoded', VectorUDT(), True), StructField('loan_encoded', VectorUDT(), True), StructField('poutcome_encoded', VectorUDT(), True), StructField('scaled_features', VectorUDT(), True), StructField('all_features', VectorUDT(), True)]. Model signature is not logged.
